## **Writeups SEKAICTF 2025 - SSSS**
Link: https://blog.sceleri.cc/posts/sekai-ctf-2025-writeup/
Chagellenge: https://2025.ctf.sekai.team/challenges/#SSSS-9

Description: Shamir SendS the Secret to everyone
```bash
ncat --ssl ssss.chals.sekai.team 1337
```

##### **Thử thách:**
Bài toán yêu cầu nhập vào một số $t$ sao cho $20 \le t \le 50$, là số bậc cao nhất của đa thức: $f(x) \equiv \sum_{i=0}^{t} c_i x^i \pmod p$. \
Với mỗi `t` vòng lặp, nhập vòa giá trị `x` và tính toán đa thức $f(x)$. \
Cuối cùng, ta nhập một số sao cho: `input == secret` thì sẽ in ra `FLAG`. Trong đó, `secret` là một hệ số ngẫu nhiên không thay đổi, vị trí hệ số thay đổi trong $f(x)$.\

##### **Ý tưởng:**
Ta chỉ có 2 vòng lặp để xác định được `secret`. Nghĩa là, ta chỉ có 2 hệ phương trình, giải 2 hệ phương trình, tìm ra 2 $f(x)$, từ đó, tìm ra `secret`.\

Nhìn chung, ta cần `degree + 1` điểm để xây dựng mỗi đa thức $f(x)$. Tuy nhiên, bài toán chỉ cung cấp `t == degree` điểm.\

Với bài toán đồng dư $p$, ta có $ \forall x, p | \gcd(x, p) = 1$, thì:
$$
x^{p-1} \equiv 1 \pmod{p}
$$

Nhiệm vụ của ta là đi tìm một số `t` sao cho $x^{t} \equiv 1 \pmod{p}$. Lại có, $p = 2^{256} - 189$ và $(p-1) \equiv 0 \pmod{29}$. Do đó, ta có thể gán $t = 29$.

Do đó, lũy thừa bậc 29 sẽ lặp lại và đồng sư với hệ số tự do, như vậy, phương trình chỉ còn 28 bậc và ta tìm được tất cả hệ số với 29 phương trình. Tức là, ta sẽ gán $g = x^{(p-1)//t}$ để $g^{29} \equiv g^{0} \pmod{p}$. Từ đó, hệ phương trình dễ dàng hạ đi một bậc.\
Mỗi vòng lặp, ta có $x_{i} = g^{i} \pmod{p}$



1. 
    ```python 
    io = process(["python3", "chall.py"])
    ```
    Khởi chạy tiến trình cục bộ và gán luồng giao tiếp qua biến `io`. Khởi chạy file và biến `io` sẽ nhận dữ liệu đầu vào.

In [ ]:
# from sage.all import *
# from pwn import *

# context.log_level = 'debug'

# p = 2 ** 256 - 189
# R = PolynomialRing(GF(p), 'x')
# t = 29

# io = process(["python3", "chall.py"])

# def sample():
#     io.sendline(str(t).encode())
#     while True:
#         g = randint(1, p)
#         g = pow(g, (p-1)//t, p)
#         if g != 1:
#             break
#     shares = []
#     for i in range(t):
#         x0 = pow(g, i, p)
#         io.sendline(str(x0).encode())
#         y0 = int(io.recvline().strip())
#         shares.append((x0, y0))
#     return R.lagrange_polynomial(shares).coefficients()

# s0 = sample()
# io.sendline(b'1')
# io.recvline()
# s1 = sample()

# for secret in set(s0) & set(s1):
#     io.sendline(str(secret).encode())
#     io.interactive()

In [19]:
from sage.all import *
from pwn import *

# context.log_level = 'debug'     # Hiển thị tất cả các thông tin chi tiết
context.log_level = 'info'     # Hiển thị cơ bản

p = 2 **256 - 189
R = PolynomialRing(GF(p), 'x') # Khởi tạo một vành đa thức 1 biến `x` trên trường hữu hạn GF(p) 
t = 29

# port = 1337
# host = 'ssss.chals.sekai.team'
# r = remote(host, port, ssl=True, sni=host)

r = process(["python3", "chall.py"])

prev_coef = []

def getFunc():
    r.sendline(str(t).encode())
    while True:
        g = randint(1, p)
        g = pow(g, (p-1)//t, p)
        if g != 1:
            break
    coordinates = []
    
    # Nhớ là a0 = a29 + a0
    for i in range(t):
        x0 = pow(g, i, p)
        r.sendline(str(x0).encode())
        y0 = int(r.recvline(keepends=False, timeout=10).strip().decode())
        coordinates.append((x0, y0))
    coefs = R.lagrange_polynomial(coordinates).coefficients()
    return coefs
    
coe1 = getFunc()
r.sendline(b'1')
r.recvline()
coe2 = getFunc()

same_coe = set(coe1) & set(coe2)
for secret in same_coe:
    r.sendline(str(secret).encode())
    flag = r.recvline(keepends=False, timeout=10).strip().decode()
    print(flag)
r.interactive()



[x] Starting local process '/usr/bin/python3'
[+] Starting local process '/usr/bin/python3': pid 27955
SEKAI{ https://youtu.be/XGxIE1hr0w4 }
[*] Switching to interactive mode
[*] Got EOF while reading in interactive
[*] Interrupted


In [11]:
index = random.randint(0, 3)
print(index)

3
